In [2]:

import os
os.chdir('../')
%pwd

'/home/jafarid/code/yogaposes'

In [45]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    resnet_base_model_path: Path
    resnet_updated_base_model_path: Path
    params_class: int
    params_image_size: list
    params_pretrained: bool
    params_type: str

In [35]:
from yogaposes.constants import *
from yogaposes.utils.common import read_yaml, create_directories


In [47]:
class ConfigurationManager: 
    def __init__(self, config_file_path= CONFIG_FILE_PATH, params_file_path= PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])

        get_prepare_base_model_config = PrepareBaseModelConfig(root_dir= config.root_dir,
                                                               resnet_base_model_path=config.resnet_base_model_path,
                                                               resnet_updated_base_model_path= config.resnet_updated_base_model_path,
                                                               params_class=self.params.CLASSES,
                                                               params_image_size= self.params.IMAGE_SIZE,
                                                               params_pretrained = self.params.PRETRAINED,
                                                               params_type= self.params.TYPE
                                                               )

        return get_prepare_base_model_config
    

In [14]:
import os
from torchvision import models
import torch 
from torchsummary import summary
from yogaposes import logger
from transformers import ViTFeatureExtractor, ViTForImageClassification
from transformers import Trainer, TrainingArguments
from torchinfo import summary

In [19]:
# Load feature extractor and model
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

/home/jafarid/miniconda3/envs/yogaposes/lib/python3.8/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [20]:
def model_summary(model):
    print("Model Summary:")
    total_params = 0
    for name, param in model.named_parameters():
        print(f"{name}: {param.numel()}")
        total_params += param.numel()
    print(f"Total Parameters: {total_params}")

model_summary(model)
model

Model Summary:
vit.embeddings.cls_token: 768
vit.embeddings.position_embeddings: 151296
vit.embeddings.patch_embeddings.projection.weight: 589824
vit.embeddings.patch_embeddings.projection.bias: 768
vit.encoder.layer.0.attention.attention.query.weight: 589824
vit.encoder.layer.0.attention.attention.query.bias: 768
vit.encoder.layer.0.attention.attention.key.weight: 589824
vit.encoder.layer.0.attention.attention.key.bias: 768
vit.encoder.layer.0.attention.attention.value.weight: 589824
vit.encoder.layer.0.attention.attention.value.bias: 768
vit.encoder.layer.0.attention.output.dense.weight: 589824
vit.encoder.layer.0.attention.output.dense.bias: 768
vit.encoder.layer.0.intermediate.dense.weight: 2359296
vit.encoder.layer.0.intermediate.dense.bias: 3072
vit.encoder.layer.0.output.dense.weight: 2359296
vit.encoder.layer.0.output.dense.bias: 768
vit.encoder.layer.0.layernorm_before.weight: 768
vit.encoder.layer.0.layernorm_before.bias: 768
vit.encoder.layer.0.layernorm_after.weight: 768
vi

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [21]:
model.classifier.out_features = 5
model

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [ ]:
model.classifier.out_features = 5
model

In [15]:

def transform_image():
    return Compose([
        Resize((224, 224)), # Resize images to fit model input size
        ToTensor(), # Convert images to tensors
        Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std) # Normalize images
    ])

train_dataset = ImageFolder('path/to/train', transform=transform_image())
val_dataset = ImageFolder('path/to/val', transform=transform_image())

train_data = ImageFolder(root=os.path.join(self.config.traning_data,'DATASET/TRAIN'), transform=composer)
val_data = ImageFolder(root=os.path.join(self.config.traning_data,'DATASET/TEST'), transform=composer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [ViTEmbeddings: 2, ViTPatchEmbeddings: 3, Conv2d: 4, Dropout: 3, ViTLayer: 4, LayerNorm: 5, ViTAttention: 5, ViTSelfAttention: 6, Linear: 7, Linear: 7, Linear: 7, Dropout: 7, ViTSelfOutput: 6, Linear: 7, Dropout: 7, LayerNorm: 5, ViTIntermediate: 5, Linear: 6, GELUActivation: 6, ViTOutput: 5, Linear: 6, Dropout: 6, ViTLayer: 4, LayerNorm: 5, ViTAttention: 5, ViTSelfAttention: 6, Linear: 7, Linear: 7, Linear: 7, Dropout: 7, ViTSelfOutput: 6, Linear: 7, Dropout: 7, LayerNorm: 5, ViTIntermediate: 5, Linear: 6, GELUActivation: 6, ViTOutput: 5, Linear: 6, Dropout: 6, ViTLayer: 4, LayerNorm: 5, ViTAttention: 5, ViTSelfAttention: 6, Linear: 7, Linear: 7, Linear: 7, Dropout: 7, ViTSelfOutput: 6, Linear: 7, Dropout: 7, LayerNorm: 5, ViTIntermediate: 5, Linear: 6, GELUActivation: 6, ViTOutput: 5, Linear: 6, Dropout: 6, ViTLayer: 4, LayerNorm: 5, ViTAttention: 5, ViTSelfAttention: 6, Linear: 7, 

In [52]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
    def get_base_model(self):
        resnet_model = models.resnet18(pretrained=self.config.params_pretrained)
        resnet_model.to(self.device)
        self.save_model(resnet_model, self.config.resnet_base_model_path)
        return resnet_model
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_till, freeze_all=True):
    # internal function that we don't want to run
        if freeze_all:
            for param in model.parameters():
                param.requires_grad = False
                
        elif (freeze_till is not None) and (freeze_till>0):
            for param in model.parameters()[:-freeze_till]:
                param.requires_grad = False
                
        n_inputs = model.fc.in_features
        model.fc = torch.nn.Linear(n_inputs, classes)
        return model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(model= self.get_base_model(), classes=self.config.params_class, freeze_all=True, freeze_till=None)
        self.full_model.to(self.device)
        summary(self.full_model,input_size=tuple(self.config.params_image_size), device=self.device)
        
        self.save_model(checkpoint=self.full_model, path=self.config.resnet_updated_base_model_path)
        logger.info(f'Saved updated model to {str(self.config.root_dir)}')

    
    @staticmethod
    def save_model(checkpoint:dict,path:Path):
        torch.save(checkpoint, path)
        

In [54]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.update_base_model()
    
except Exception as e:
    raise e

[2024-03-02 17:54:21,305: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-02 17:54:21,306: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-02 17:54:21,307: INFO: common: created directory at: artifacts]
[2024-03-02 17:54:21,307: INFO: common: created directory at: artifacts/prepare_base_model]
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-